In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [3]:
var = "solidprcptot"

In [4]:
#Variable 
variable85="rcp85_"+var+"_monthly"
variable45="rcp45_"+var+"_monthly"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM

In [5]:
monthlist = ["january", "february", "march", "april", "may", "june", "july", "august", "september", 
             "october", "november", "december"]

# FOR RCP 45

## Subsetting arrays by periods of time with Xclim

In [6]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.month[0:12]
# Monthly example 30 y means
listds = []
i=0
for s in dsEns45.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds.append(tmp1)

0
<xarray.Dataset>
Dimensions:       (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * time          (time) datetime64[ns] 1981-01-01 1982-01-01 ... 2100-01-01
  * lon           (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * lat           (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * realization   (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    solidprcptot  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:       (lat: 320, lo

## Loop to create all 12 dataframes

In [7]:
i = 0
df45list = []
for ds in listds:
    print ("Dataset number: ", i, "--------------")
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df45=0
    df453 = df452.loc[df452["realization"] == 0].dropna()
    df452=0
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df453.groupby([df453.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp452 =  dfp.reset_index()
    dfp=0
    print ("Split in 3 different time periods")
    df45C = dfp452.copy()
    df45C[var+"_p10"] = round(df45C[var+"_p10"]/10, 2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"]/10, 2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"]/10, 2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    df45C=0
    print ("pivoted")
    df45f = Region1i45.pivot_table(index=["lat","lon"], columns="time")
    Region1i45=0
    print ("DF pivoted")
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"    
    print (month)         
    df45f.columns = [month + "_" + year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df45list.append(df45f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
january
Columns renamed
DF added to the list
Dataset number:  1 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
february
Columns renamed
DF added to the list
Dataset number:  2 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
march
Columns renamed
DF added to the list
Dataset number:  3 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
april
Columns renamed
DF added to the list
Dataset number:  4 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
may
Columns renamed
DF 

In [8]:
df45list


[                      january_hist_rcp45_p10  january_t2050_rcp45_p10  \
 lat       lon                                                           
 40.041039 -89.045212                    0.64                     0.35   
           -88.961884                    0.65                     0.37   
           -88.878548                    0.68                     0.37   
           -88.795219                    0.69                     0.37   
           -88.711891                    0.68                     0.37   
 ...                                      ...                      ...   
 66.623306 -61.629639                    1.20                     1.53   
           -61.546310                    1.25                     1.59   
           -61.462978                    1.22                     1.56   
           -61.379650                    1.21                     1.55   
           -61.296322                    1.10                     1.42   
 
                       january_t2080

In [9]:
def merge_multiple_dfs(df_list, on):
    df_result = df_list[0]
    for df in df_list[1:]:
        df_result = df_result.merge(df, on=on)
    return df_result

In [10]:
mkey = ["lat", "lon"]

In [11]:
dfse45 = merge_multiple_dfs(df45list[0:12], on=mkey)

In [12]:
dfse452 = dfse45.reset_index()
dfse45=0

In [13]:
dfse452.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_monthly.feather")

In [14]:
dfse452 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_monthly.feather")

In [15]:
dfse453 = dfse452.drop(columns=["january_hist_rcp45_p10","january_hist_rcp45_p50", "january_hist_rcp45_p90", 
                               "february_hist_rcp45_p10", "february_hist_rcp45_p50", "february_hist_rcp45_p90", 
                               "march_hist_rcp45_p10","march_hist_rcp45_p50", "march_hist_rcp45_p90", 
                               "april_hist_rcp45_p10","april_hist_rcp45_p50", "april_hist_rcp45_p90",
                              "may_hist_rcp45_p10","may_hist_rcp45_p50", "may_hist_rcp45_p90", 
                               "june_hist_rcp45_p10", "june_hist_rcp45_p50", "june_hist_rcp45_p90", 
                               "july_hist_rcp45_p10","july_hist_rcp45_p50", "july_hist_rcp45_p90", 
                               "august_hist_rcp45_p10","august_hist_rcp45_p50", "august_hist_rcp45_p90",
                              "september_hist_rcp45_p10","september_hist_rcp45_p50", "september_hist_rcp45_p90", 
                               "october_hist_rcp45_p10", "october_hist_rcp45_p50", "october_hist_rcp45_p90", 
                               "november_hist_rcp45_p10","november_hist_rcp45_p50", "november_hist_rcp45_p90", 
                               "december_hist_rcp45_p10","december_hist_rcp45_p50", "december_hist_rcp45_p90"])
dfse453


,lat,lon,january_t2050_rcp45_p10,january_t2080_rcp45_p10,january_t2050_rcp45_p50,january_t2080_rcp45_p50,january_t2050_rcp45_p90,january_t2080_rcp45_p90,february_t2050_rcp45_p10,february_t2080_rcp45_p10,...,november_t2050_rcp45_p50,november_t2080_rcp45_p50,november_t2050_rcp45_p90,november_t2080_rcp45_p90,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,0.35,0.28,1.40,1.20,3.25,3.16,0.18,0.11,...,0.02,0.01,0.64,0.52,0.10,0.10,1.05,0.82,3.27,3.29
1,40.041039,-88.961884,0.37,0.28,1.43,1.24,3.35,3.20,0.19,0.12,...,0.02,0.01,0.65,0.51,0.10,0.10,1.07,0.82,3.31,3.29
2,40.041039,-88.878548,0.37,0.28,1.47,1.28,3.38,3.23,0.18,0.12,...,0.02,0.01,0.64,0.51,0.11,0.11,1.10,0.84,3.37,3.34
3,40.041039,-88.795219,0.37,0.28,1.48,1.28,3.42,3.24,0.19,0.13,...,0.02,0.01,0.60,0.50,0.11,0.11,1.10,0.88,3.34,3.31
4,40.041039,-88.711891,0.37,0.28,1.49,1.30,3.48,3.27,0.18,0.12,...,0.02,0.01,0.55,0.46,0.10,0.11,1.10,0.88,3.36,3.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,1.53,1.32,3.73,3.37,8.44,8.09,0.90,1.17,...,5.38,5.23,9.89,9.23,1.99,2.13,4.55,4.86,8.92,8.89
76202,66.623306,-61.546310,1.59,1.37,3.81,3.46,8.55,8.19,0.95,1.24,...,5.51,5.40,10.06,9.45,2.09,2.21,4.65,4.98,9.02,9.07
76203,66.623306,-61.462978,1.56,1.33,3.77,3.42,8.47,8.16,0.92,1.20,...,5.43,5.30,9.90,9.35,2.06,2.17,4.62,4.91,8.96,9.02
76204,66.623306,-61.379650,1.55,1.31,3.74,3.40,8.41,8.11,0.90,1.18,...,5.38,5.29,9.81,9.27,2.06,2.15,4.58,4.89,8.88,8.95


In [16]:
lisdfmoh45 = []
for month in monthlist: 
    dfmoh = dfse452[["lat", "lon", month+"_hist_rcp45_p50"]]
    print (month)
    lisdfmoh45.append(dfmoh)

january
february
march
april
may
june
july
august
september
october
november
december


# FOR RCP 85

## Subsetting arrays by periods of time with Xclim

In [17]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.month[0:12]
# Monthly example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

0
<xarray.Dataset>
Dimensions:       (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * time          (time) datetime64[ns] 1981-01-01 1982-01-01 ... 2100-01-01
  * lon           (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * lat           (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * realization   (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    solidprcptot  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:       (lat: 320, lo

## Loop to create all 12 dataframes

In [18]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df85=0
    df853 = df852.loc[df852["realization"] == 0].dropna()
    df852=0
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df853.groupby([df853.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp852 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df85C = dfp852.copy()
    dfp852=0
    df85C[var+"_p10"] = round(df85C[var+"_p10"]/10, 2)
    df85C[var+"_p50"] = round(df85C[var+"_p50"]/10, 2)
    df85C[var+"_p90"] = round(df85C[var+"_p90"]/10, 2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    print ("pivoted")
    df85f = Region1i85.pivot_table(index=["lat","lon"], columns="time")
    Region1i85=0
    print ("DF pivoted")
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"    
    print (month)
    df85f.columns = [month + "_" + year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df85list.append(df85f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
january
Columns renamed
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
february
Columns renamed
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
march
Columns renamed
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
april
Columns renamed
DF added to the list
Dataset number:  4
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
may
Columns renamed
DF added to the list
Dataset number:  5
Percentiles Obteined
Converted to DF
S

In [19]:
df85list


[                      january_hist_rcp85_p10  january_t2050_rcp85_p10  \
 lat       lon                                                           
 40.041039 -89.045212                    0.63                     0.29   
           -88.961884                    0.65                     0.29   
           -88.878548                    0.68                     0.30   
           -88.795219                    0.69                     0.31   
           -88.711891                    0.68                     0.31   
 ...                                      ...                      ...   
 66.623306 -61.629639                    1.18                     1.60   
           -61.546310                    1.23                     1.66   
           -61.462978                    1.20                     1.61   
           -61.379650                    1.19                     1.60   
           -61.296322                    1.08                     1.45   
 
                       january_t2080

In [20]:
dfse85 = merge_multiple_dfs(df85list[0:12], on=mkey).reset_index()

In [21]:
dfse85

,lat,lon,january_hist_rcp85_p10,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_hist_rcp85_p50,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_hist_rcp85_p90,january_t2050_rcp85_p90,...,november_t2080_rcp85_p90,december_hist_rcp85_p10,december_t2050_rcp85_p10,december_t2080_rcp85_p10,december_hist_rcp85_p50,december_t2050_rcp85_p50,december_t2080_rcp85_p50,december_hist_rcp85_p90,december_t2050_rcp85_p90,december_t2080_rcp85_p90
0,40.041039,-89.045212,0.63,0.29,0.10,1.87,1.28,0.80,3.75,3.27,...,0.12,0.49,0.06,0.02,1.57,0.75,0.45,4.08,2.67,2.32
1,40.041039,-88.961884,0.65,0.29,0.11,1.92,1.31,0.80,3.83,3.41,...,0.12,0.47,0.05,0.02,1.60,0.77,0.46,4.17,2.67,2.31
2,40.041039,-88.878548,0.68,0.30,0.11,1.93,1.35,0.79,3.87,3.44,...,0.12,0.48,0.05,0.02,1.64,0.79,0.47,4.20,2.66,2.35
3,40.041039,-88.795219,0.69,0.31,0.11,2.00,1.35,0.81,3.83,3.40,...,0.13,0.53,0.06,0.02,1.66,0.79,0.49,4.13,2.69,2.35
4,40.041039,-88.711891,0.68,0.31,0.11,2.02,1.35,0.80,3.90,3.42,...,0.13,0.53,0.05,0.02,1.70,0.78,0.49,4.20,2.74,2.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,1.18,1.60,1.51,2.76,3.72,4.13,6.92,8.30,...,10.60,1.45,2.03,1.37,3.79,4.55,4.29,7.51,9.10,8.84
76202,66.623306,-61.546310,1.23,1.66,1.60,2.83,3.82,4.24,6.99,8.42,...,10.81,1.51,2.11,1.53,3.87,4.76,4.47,7.60,9.25,8.96
76203,66.623306,-61.462978,1.20,1.61,1.52,2.77,3.76,4.21,6.86,8.38,...,10.68,1.48,2.09,1.49,3.86,4.71,4.38,7.57,9.20,8.88
76204,66.623306,-61.379650,1.19,1.60,1.50,2.76,3.73,4.19,6.80,8.30,...,10.61,1.48,2.05,1.47,3.83,4.70,4.35,7.54,9.14,8.80


In [22]:
dfse85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_monthly.feather")

In [23]:
dfse85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_monthly.feather")

In [24]:
dfse852 = dfse85.drop(columns=["january_hist_rcp85_p10","january_hist_rcp85_p50", "january_hist_rcp85_p90", 
                               "february_hist_rcp85_p10", "february_hist_rcp85_p50", "february_hist_rcp85_p90", 
                               "march_hist_rcp85_p10","march_hist_rcp85_p50", "march_hist_rcp85_p90", 
                               "april_hist_rcp85_p10","april_hist_rcp85_p50", "april_hist_rcp85_p90",
                              "may_hist_rcp85_p10","may_hist_rcp85_p50", "may_hist_rcp85_p90", 
                               "june_hist_rcp85_p10", "june_hist_rcp85_p50", "june_hist_rcp85_p90", 
                               "july_hist_rcp85_p10","july_hist_rcp85_p50", "july_hist_rcp85_p90", 
                               "august_hist_rcp85_p10","august_hist_rcp85_p50", "august_hist_rcp85_p90",
                              "september_hist_rcp85_p10","september_hist_rcp85_p50", "september_hist_rcp85_p90", 
                               "october_hist_rcp85_p10", "october_hist_rcp85_p50", "october_hist_rcp85_p90", 
                               "november_hist_rcp85_p10","november_hist_rcp85_p50", "november_hist_rcp85_p90", 
                               "december_hist_rcp85_p10","december_hist_rcp85_p50", "december_hist_rcp85_p90"])
dfse852


,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2050_rcp85_p50,november_t2080_rcp85_p50,november_t2050_rcp85_p90,november_t2080_rcp85_p90,december_t2050_rcp85_p10,december_t2080_rcp85_p10,december_t2050_rcp85_p50,december_t2080_rcp85_p50,december_t2050_rcp85_p90,december_t2080_rcp85_p90
0,40.041039,-89.045212,0.29,0.10,1.28,0.80,3.27,2.49,0.11,0.00,...,0.01,0.00,0.42,0.12,0.06,0.02,0.75,0.45,2.67,2.32
1,40.041039,-88.961884,0.29,0.11,1.31,0.80,3.41,2.55,0.11,0.00,...,0.01,0.00,0.44,0.12,0.05,0.02,0.77,0.46,2.67,2.31
2,40.041039,-88.878548,0.30,0.11,1.35,0.79,3.44,2.55,0.11,0.01,...,0.01,0.00,0.43,0.12,0.05,0.02,0.79,0.47,2.66,2.35
3,40.041039,-88.795219,0.31,0.11,1.35,0.81,3.40,2.57,0.12,0.00,...,0.00,0.00,0.43,0.13,0.06,0.02,0.79,0.49,2.69,2.35
4,40.041039,-88.711891,0.31,0.11,1.35,0.80,3.42,2.56,0.11,0.00,...,0.00,0.00,0.40,0.13,0.05,0.02,0.78,0.49,2.74,2.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,1.60,1.51,3.72,4.13,8.30,8.66,1.09,1.29,...,5.01,5.02,9.46,10.60,2.03,1.37,4.55,4.29,9.10,8.84
76202,66.623306,-61.546310,1.66,1.60,3.82,4.24,8.42,8.89,1.16,1.35,...,5.13,5.28,9.58,10.81,2.11,1.53,4.76,4.47,9.25,8.96
76203,66.623306,-61.462978,1.61,1.52,3.76,4.21,8.38,8.73,1.11,1.30,...,5.01,5.16,9.49,10.68,2.09,1.49,4.71,4.38,9.20,8.88
76204,66.623306,-61.379650,1.60,1.50,3.73,4.19,8.30,8.69,1.09,1.24,...,4.95,5.04,9.40,10.61,2.05,1.47,4.70,4.35,9.14,8.80


In [25]:
lisdfmoh85 = []
for month in monthlist: 
    dfmoh = dfse85[["lat", "lon", month+"_hist_rcp85_p50"]]
    print (month)
    lisdfmoh85.append(dfmoh)

january
february
march
april
may
june
july
august
september
october
november
december


In [26]:
lisdfmoh85

[             lat        lon  january_hist_rcp85_p50
 0      40.041039 -89.045212                    1.87
 1      40.041039 -88.961884                    1.92
 2      40.041039 -88.878548                    1.93
 3      40.041039 -88.795219                    2.00
 4      40.041039 -88.711891                    2.02
 ...          ...        ...                     ...
 76201  66.623306 -61.629639                    2.76
 76202  66.623306 -61.546310                    2.83
 76203  66.623306 -61.462978                    2.77
 76204  66.623306 -61.379650                    2.76
 76205  66.623306 -61.296322                    2.58
 
 [76206 rows x 3 columns],              lat        lon  february_hist_rcp85_p50
 0      40.041039 -89.045212                     1.39
 1      40.041039 -88.961884                     1.43
 2      40.041039 -88.878548                     1.44
 3      40.041039 -88.795219                     1.45
 4      40.041039 -88.711891                     1.38
 ...        

# Merge df45 and 85

# Extract hist

## Getting historic mean for each month

In [27]:
### Function
def mean_month(month, dfse45, dfse85):
    print(month)
    dfw45 = dfse45[["lat", "lon", month+"_hist_rcp45_p50"]]
    dfw85 = dfse85[["lat", "lon", month+"_hist_rcp85_p50"]]
    dfsew = pd.merge(dfw45, dfw85, on=["lat","lon"])
    dfw45=0
    dfw85=0
    dfsew[month+"_hist_p50"] = round((dfsew[month+"_hist_rcp45_p50"]+ dfsew[month+"_hist_rcp85_p50"])/2, 2)
    dftgwh2 = dfsew.reset_index()
    dfsew=0
    dftgwh3 = dftgwh2[["lat", "lon", month+"_hist_p50"]]
    dftgwh2=0
    return (dftgwh3)


### All months historic mean

In [28]:
listdfmo = []
i=0
for month in monthlist:
    print(month)
    dfm = mean_month(month, lisdfmoh45[i], lisdfmoh85[i])
    i = i+1
    listdfmo.append(dfm)

january
january
february
february
march
march
april
april
may
may
june
june
july
july
august
august
september
september
october
october
november
november
december
december


# Merge means with the 45 and 85 DF

In [29]:
dftgseall = dfse852.merge(dfse452, on=mkey)
i=0
for df in listdfmo:
    dftgseall2 = dftgseall.merge(listdfmo[i], on=mkey)
    i = i +1


In [30]:
dftgseall

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2080_rcp45_p90,december_hist_rcp45_p10,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_hist_rcp45_p50,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_hist_rcp45_p90,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,0.29,0.10,1.28,0.80,3.27,2.49,0.11,0.00,...,0.52,0.46,0.10,0.10,1.51,1.05,0.82,4.09,3.27,3.29
1,40.041039,-88.961884,0.29,0.11,1.31,0.80,3.41,2.55,0.11,0.00,...,0.51,0.46,0.10,0.10,1.52,1.07,0.82,4.21,3.31,3.29
2,40.041039,-88.878548,0.30,0.11,1.35,0.79,3.44,2.55,0.11,0.01,...,0.51,0.48,0.11,0.11,1.56,1.10,0.84,4.27,3.37,3.34
3,40.041039,-88.795219,0.31,0.11,1.35,0.81,3.40,2.57,0.12,0.00,...,0.50,0.49,0.11,0.11,1.57,1.10,0.88,4.18,3.34,3.31
4,40.041039,-88.711891,0.31,0.11,1.35,0.80,3.42,2.56,0.11,0.00,...,0.46,0.48,0.10,0.11,1.56,1.10,0.88,4.21,3.36,3.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,1.60,1.51,3.72,4.13,8.30,8.66,1.09,1.29,...,9.23,1.56,1.99,2.13,3.98,4.55,4.86,7.88,8.92,8.89
76202,66.623306,-61.546310,1.66,1.60,3.82,4.24,8.42,8.89,1.16,1.35,...,9.45,1.64,2.09,2.21,4.06,4.65,4.98,7.99,9.02,9.07
76203,66.623306,-61.462978,1.61,1.52,3.76,4.21,8.38,8.73,1.11,1.30,...,9.35,1.62,2.06,2.17,3.98,4.62,4.91,7.92,8.96,9.02
76204,66.623306,-61.379650,1.60,1.50,3.73,4.19,8.30,8.69,1.09,1.24,...,9.27,1.61,2.06,2.15,3.97,4.58,4.89,7.87,8.88,8.95


In [31]:
dfmop50 = merge_multiple_dfs(listdfmo[0:12], on=mkey)

In [32]:
dfmop50

,lat,lon,january_hist_p50,february_hist_p50,march_hist_p50,april_hist_p50,may_hist_p50,june_hist_p50,july_hist_p50,august_hist_p50,september_hist_p50,october_hist_p50,november_hist_p50,december_hist_p50
0,40.041039,-89.045212,1.87,1.38,0.40,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.20,1.54
1,40.041039,-88.961884,1.92,1.42,0.42,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.20,1.56
2,40.041039,-88.878548,1.94,1.43,0.43,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.22,1.60
3,40.041039,-88.795219,1.98,1.44,0.44,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.22,1.62
4,40.041039,-88.711891,2.01,1.39,0.43,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.20,1.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,2.83,2.31,2.30,2.78,2.91,0.20,0.0,0.0,0.97,5.59,4.66,3.88
76202,66.623306,-61.546310,2.90,2.38,2.40,2.85,3.12,0.26,0.0,0.0,1.16,5.82,4.76,3.96
76203,66.623306,-61.462978,2.84,2.34,2.34,2.82,2.96,0.22,0.0,0.0,1.02,5.70,4.69,3.92
76204,66.623306,-61.379650,2.82,2.29,2.32,2.80,2.94,0.21,0.0,0.0,0.97,5.61,4.64,3.90


In [33]:
dftgseall2 = dfse852.merge(dfse452, on=mkey)

In [34]:
#dftgseall = dfse852.merge(dfse452, on=mkey).merge(dftfah, on=mkey).merge(dftsuh, on=mkey).merge(dftsph, on=mkey).merge(dftgwh, on=mkey)

In [35]:
dftgseall2

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2080_rcp45_p90,december_hist_rcp45_p10,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_hist_rcp45_p50,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_hist_rcp45_p90,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,0.29,0.10,1.28,0.80,3.27,2.49,0.11,0.00,...,0.52,0.46,0.10,0.10,1.51,1.05,0.82,4.09,3.27,3.29
1,40.041039,-88.961884,0.29,0.11,1.31,0.80,3.41,2.55,0.11,0.00,...,0.51,0.46,0.10,0.10,1.52,1.07,0.82,4.21,3.31,3.29
2,40.041039,-88.878548,0.30,0.11,1.35,0.79,3.44,2.55,0.11,0.01,...,0.51,0.48,0.11,0.11,1.56,1.10,0.84,4.27,3.37,3.34
3,40.041039,-88.795219,0.31,0.11,1.35,0.81,3.40,2.57,0.12,0.00,...,0.50,0.49,0.11,0.11,1.57,1.10,0.88,4.18,3.34,3.31
4,40.041039,-88.711891,0.31,0.11,1.35,0.80,3.42,2.56,0.11,0.00,...,0.46,0.48,0.10,0.11,1.56,1.10,0.88,4.21,3.36,3.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,1.60,1.51,3.72,4.13,8.30,8.66,1.09,1.29,...,9.23,1.56,1.99,2.13,3.98,4.55,4.86,7.88,8.92,8.89
76202,66.623306,-61.546310,1.66,1.60,3.82,4.24,8.42,8.89,1.16,1.35,...,9.45,1.64,2.09,2.21,4.06,4.65,4.98,7.99,9.02,9.07
76203,66.623306,-61.462978,1.61,1.52,3.76,4.21,8.38,8.73,1.11,1.30,...,9.35,1.62,2.06,2.17,3.98,4.62,4.91,7.92,8.96,9.02
76204,66.623306,-61.379650,1.60,1.50,3.73,4.19,8.30,8.69,1.09,1.24,...,9.27,1.61,2.06,2.15,3.97,4.58,4.89,7.87,8.88,8.95


In [36]:
dftgseall3 = dftgseall2.merge(dfmop50, on=mkey)
dftgseall2=0

In [37]:
dftgseall3.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_monthly.feather")

In [38]:
dftgseall3

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,march_hist_p50,april_hist_p50,may_hist_p50,june_hist_p50,july_hist_p50,august_hist_p50,september_hist_p50,october_hist_p50,november_hist_p50,december_hist_p50
0,40.041039,-89.045212,0.29,0.10,1.28,0.80,3.27,2.49,0.11,0.00,...,0.40,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.20,1.54
1,40.041039,-88.961884,0.29,0.11,1.31,0.80,3.41,2.55,0.11,0.00,...,0.42,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.20,1.56
2,40.041039,-88.878548,0.30,0.11,1.35,0.79,3.44,2.55,0.11,0.01,...,0.43,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.22,1.60
3,40.041039,-88.795219,0.31,0.11,1.35,0.81,3.40,2.57,0.12,0.00,...,0.44,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.22,1.62
4,40.041039,-88.711891,0.31,0.11,1.35,0.80,3.42,2.56,0.11,0.00,...,0.43,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.20,1.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,1.60,1.51,3.72,4.13,8.30,8.66,1.09,1.29,...,2.30,2.78,2.91,0.20,0.0,0.0,0.97,5.59,4.66,3.88
76202,66.623306,-61.546310,1.66,1.60,3.82,4.24,8.42,8.89,1.16,1.35,...,2.40,2.85,3.12,0.26,0.0,0.0,1.16,5.82,4.76,3.96
76203,66.623306,-61.462978,1.61,1.52,3.76,4.21,8.38,8.73,1.11,1.30,...,2.34,2.82,2.96,0.22,0.0,0.0,1.02,5.70,4.69,3.92
76204,66.623306,-61.379650,1.60,1.50,3.73,4.19,8.30,8.69,1.09,1.24,...,2.32,2.80,2.94,0.21,0.0,0.0,0.97,5.61,4.64,3.90


# Merge complete DF with Polygons for each spatial scale

## Dictionary - regions: column name, short name

In [39]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [40]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [41]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [42]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall3, dfpolyshape, on=["lat","lon"])
    dfpolyshape = 0
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_monthly.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_monthly.json", driver="GeoJSON")
    dftp = 0
    gdf = 0

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_solidprcptot_monthly.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
